# Fake News Detection




## Objective


The objective of this assignment is to develop a Semantic Classification model. You will be using Word2Vec method to extract the semantic relations from the text and develop a basic understanding of how to train supervised models to categorise text based on its meaning, rather than just syntax. You will explore how this technique is used in situations where understanding textual meaning plays a critical role in making accurate and efficient decisions.


## Business Objective

The spread of fake news has become a significant challenge in today’s digital world. With the massive volume of news articles published daily, it’s becoming harder to distinguish between credible and misleading information. This creates a need for systems that can automatically classify news articles as true or fake, helping to reduce misinformation and protect public trust.


In this assignment, you will develop a Semantic Classification model that uses the Word2Vec method to detect recurring patterns and themes in news articles. Using supervised learning models, the goal is to build a system that classifies news articles as either fake or true.


<h2> Pipelines that needs to be performed </h2>

You need to perform the following tasks to complete the assignment:

<ol type="1">

  <li> Data Preparation
  <li> Text Preprocessing
  <li> Train Validation Split
  <li> EDA on Training Data
  <li> EDA on Validation Data [Optional]
  <li> Feature Extraction
  <li> Model Training and Evaluation

</ol>

---

**NOTE:** The marks given along with headings and sub-headings are cumulative marks for those particular headings/sub-headings.<br>

The actual marks for each task are specified within the tasks themselves.

For example, marks given with heading *2* or sub-heading *2.1* are the cumulative marks, for your reference only. <br>

The marks you will receive for completing tasks are given with the tasks.

Suppose the marks for two tasks are: 3 marks for 2.1.1 and 2 marks for 3.2.2, or
* 2.1.1 [3 marks]
* 3.2.2 [2 marks]

then, you will earn 3 marks for completing task 2.1.1 and 2 marks for completing task 3.2.2.


---

## Data Dictionary


For this assignment, you will work with two datasets, `True.csv` and `Fake.csv`.
Both datasets contain three columns:
<ul>
  <li> title of the news article
  <li> text of the news article
  <li> date of article publication
</ul>

`True.csv` dataset includes 21,417 true news, while the `Fake.csv` dataset comprises 23,502 fake news.

## Installing required Libraries

In [1]:
# !pip install --upgrade numpy==1.26.4
# !pip install --upgrade pandas==2.2.2
# !pip install --upgrade nltk==3.9.1
# !pip install --upgrade spacy==3.7.5
# !pip install --upgrade scipy==1.12
# !pip install --upgrade pydantic==2.10.5
# !pip install wordcloud==1.9.4
# !python -m spacy download en_core_web_sm

## Importing the necessary libraries

In [1]:
# Import essential libraries for data manipulation and analysis
import numpy as np  # For numerical operations and arrays
import pandas as pd  # For working with dataframes and structured data
import re  # For regular expression operations (text processing)
import nltk  # Natural Language Toolkit for text processing
import spacy  # For advanced NLP tasks
import string  # For handling string-related operations/'

# Optional: Uncomment the line below to enable GPU support for spaCy (if you have a compatible GPU)
# spacy.require_gpu()

# Load the spaCy small English language model
nlp = spacy.load("en_core_web_sm")

# For data visualization
import seaborn as sns  # Data visualization library for statistical graphics
import matplotlib.pyplot as plt  # Matplotlib for creating static plots

# Configure Matplotlib to display plots inline in Jupyter Notebook


# Suppress unnecessary warnings to keep output clean
import warnings

warnings.filterwarnings("ignore")

# For interactive plots
from plotly.offline import plot  # Enables offline plotting with Plotly
import plotly.graph_objects as go  # For creating customizable Plotly plots
import plotly.express as px  # A high-level interface for Plotly

# For preprocessing and feature extraction in machine learning
from sklearn.feature_extraction.text import (  # Methods for text vectorization
    CountVectorizer,  # Converts text into a bag-of-words model
)

# Import accuracy, precision, recall, f_score from sklearn to predict train accuracy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Pretty printing for better readability of output
from pprint import pprint

# For progress tracking in loops (useful for larger datasets)
from tqdm import tqdm, tqdm_notebook  # Progress bar for loops

tqdm.pandas()  # Enables progress bars for pandas operations

In [3]:
## Change the display properties of pandas to max
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

## Load the data

Load the True.csv and Fake.csv files as DataFrames

In [4]:
# Import the first file - True.csv
true = pd.read_excel("True.xlsx")
# Import the second file - Fake.csv
fake = pd.read_excel("Fake.xlsx")

In [2]:
from IPython.display import display


def display_info(df):
    display(df[:10])
    print("\n")
    display(df.info())

In [6]:
display_info(true)

title  \
0                In Republican state, Obama talks with worried American mom   
1  South Korea calls for 'bone-numbing' sanctions on North for nuclear test   
2     U.S. to expand refugee program for Central Americans fleeing violence   
3    Bid to mine more coal on U.S. federal lands tests Obama's green agenda   
4              For Republican Rubio, a moment of truth in race to lead U.S.   
5                       Cheesy puns abound as White House hosts online chat   
6            New York's Cuomo proposes $145.3 billion budget, caps spending   
7                         Trump looms behind both Obama and Haley speeches    
8         House of Representatives to try again to seek Iran deal clampdown   
9                                      Obama knocks Trump, voices optimism    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   title   21417 non-null  object        
 1   text    21417 non-null  object        
 2   date    21417 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 502.1+ KB


None

In [7]:
display_info(fake)

title  \
0                  Carly Fiorina‚Äôs Poor Attempt To Pander To Iowa Football Fans Goes Horribly Wrong (TWEETS)   
1   Jeb! Makes Awkward ‚ÄòHurricane Katrina‚Äô Joke, Reminds Voters Of Brother‚Äôs Awful Record (VIDEO/TWEETS)   
2                          R.I.P. GOP White House Dreams? U.S. Jobs Haven‚Äôt Grown This Much Since The ‚Äò90s   
3                                   Guess Who Donald Trump Blames For His Ridiculous Orange Clown Hair (VIDEO)   
4                              Christian Fundamentalist A**hole Leaves Waiter A Dirty Trick For A Tip (IMAGES)   
5                   Ted Cruz Tells His Fanatics To Put On Their ‚ÄòArmor Of God‚Äô To Prepare To Fight For Him   
6                    Road Raging Texas ‚ÄòGood Guy With A Gun‚Äô Shoots Woman In The Head, Leaves Her For Dead   
7                 Heartbreaking: This Democratic Senator Just Tweeted The Entire Mass Shooting History Of 2015   
8                                        Cenk Uygur Rages At White Apathy After CPD Murders Black Teen (VIDEO)   
9                            Hillary Was Right: Trump Rant On Muslims Shows Up On New Terror Recruitment Video   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23539 entries, 0 to 23538
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   23499 non-null  object
 1   text    23499 non-null  object
 2   date    23479 non-null  object
dtypes: object(3)
memory usage: 551.8+ KB


None

## **1.** Data Preparation  <font color = red>[10 marks]</font>





### **1.0** Data Understanding

In [8]:
fake["date"].value_counts(ascending=True)[:10]

date
 Mercy Corps places all blame for Palestinian poverty and suffering directly on Israel.Mexican American Legal Defense and Education Fund: This group advocates open borders    1
2015-07-19 00:00:00                                                                                                                                                            1
2018-02-14 00:00:00                                                                                                                                                            1
2015-06-21 00:00:00                                                                                                                                                            1
2015-04-02 00:00:00                                                                                                                                                            1
MSNBC HOST Rudely Assumes Steel Worker Would Never Let His Son Follow in His Footsteps‚Ä¶He Couldn‚Äôt Be More

In [9]:
# Inspect the DataFrame with True News to understand the given data
display_info(true)

title  \
0                In Republican state, Obama talks with worried American mom   
1  South Korea calls for 'bone-numbing' sanctions on North for nuclear test   
2     U.S. to expand refugee program for Central Americans fleeing violence   
3    Bid to mine more coal on U.S. federal lands tests Obama's green agenda   
4              For Republican Rubio, a moment of truth in race to lead U.S.   
5                       Cheesy puns abound as White House hosts online chat   
6            New York's Cuomo proposes $145.3 billion budget, caps spending   
7                         Trump looms behind both Obama and Haley speeches    
8         House of Representatives to try again to seek Iran deal clampdown   
9                                      Obama knocks Trump, voices optimism    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   title   21417 non-null  object        
 1   text    21417 non-null  object        
 2   date    21417 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 502.1+ KB


None

In [10]:
# Inspect the DataFrame with Fake News to understand the given data
display_info(fake)

title  \
0                  Carly Fiorina‚Äôs Poor Attempt To Pander To Iowa Football Fans Goes Horribly Wrong (TWEETS)   
1   Jeb! Makes Awkward ‚ÄòHurricane Katrina‚Äô Joke, Reminds Voters Of Brother‚Äôs Awful Record (VIDEO/TWEETS)   
2                          R.I.P. GOP White House Dreams? U.S. Jobs Haven‚Äôt Grown This Much Since The ‚Äò90s   
3                                   Guess Who Donald Trump Blames For His Ridiculous Orange Clown Hair (VIDEO)   
4                              Christian Fundamentalist A**hole Leaves Waiter A Dirty Trick For A Tip (IMAGES)   
5                   Ted Cruz Tells His Fanatics To Put On Their ‚ÄòArmor Of God‚Äô To Prepare To Fight For Him   
6                    Road Raging Texas ‚ÄòGood Guy With A Gun‚Äô Shoots Woman In The Head, Leaves Her For Dead   
7                 Heartbreaking: This Democratic Senator Just Tweeted The Entire Mass Shooting History Of 2015   
8                                        Cenk Uygur Rages At White Apathy After CPD Murders Black Teen (VIDEO)   
9                            Hillary Was Right: Trump Rant On Muslims Shows Up On New Terror Recruitment Video   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23539 entries, 0 to 23538
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   23499 non-null  object
 1   text    23499 non-null  object
 2   date    23479 non-null  object
dtypes: object(3)
memory usage: 551.8+ KB


None

In [11]:
# Print the column details for True News DataFrame
display_info(true)

title  \
0                In Republican state, Obama talks with worried American mom   
1  South Korea calls for 'bone-numbing' sanctions on North for nuclear test   
2     U.S. to expand refugee program for Central Americans fleeing violence   
3    Bid to mine more coal on U.S. federal lands tests Obama's green agenda   
4              For Republican Rubio, a moment of truth in race to lead U.S.   
5                       Cheesy puns abound as White House hosts online chat   
6            New York's Cuomo proposes $145.3 billion budget, caps spending   
7                         Trump looms behind both Obama and Haley speeches    
8         House of Representatives to try again to seek Iran deal clampdown   
9                                      Obama knocks Trump, voices optimism    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   title   21417 non-null  object        
 1   text    21417 non-null  object        
 2   date    21417 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 502.1+ KB


None

In [12]:
# Print the column details for Fake News Dataframe
display_info(fake)

title  \
0                  Carly Fiorina‚Äôs Poor Attempt To Pander To Iowa Football Fans Goes Horribly Wrong (TWEETS)   
1   Jeb! Makes Awkward ‚ÄòHurricane Katrina‚Äô Joke, Reminds Voters Of Brother‚Äôs Awful Record (VIDEO/TWEETS)   
2                          R.I.P. GOP White House Dreams? U.S. Jobs Haven‚Äôt Grown This Much Since The ‚Äò90s   
3                                   Guess Who Donald Trump Blames For His Ridiculous Orange Clown Hair (VIDEO)   
4                              Christian Fundamentalist A**hole Leaves Waiter A Dirty Trick For A Tip (IMAGES)   
5                   Ted Cruz Tells His Fanatics To Put On Their ‚ÄòArmor Of God‚Äô To Prepare To Fight For Him   
6                    Road Raging Texas ‚ÄòGood Guy With A Gun‚Äô Shoots Woman In The Head, Leaves Her For Dead   
7                 Heartbreaking: This Democratic Senator Just Tweeted The Entire Mass Shooting History Of 2015   
8                                        Cenk Uygur Rages At White Apathy After CPD Murders Black Teen (VIDEO)   
9                            Hillary Was Right: Trump Rant On Muslims Shows Up On New Terror Recruitment Video   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23539 entries, 0 to 23538
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   23499 non-null  object
 1   text    23499 non-null  object
 2   date    23479 non-null  object
dtypes: object(3)
memory usage: 551.8+ KB


None

In [13]:
# Print the column names of both DataFrames
print("True columns")
for col in true.columns:
    print(col, end="\t")
print("\nFake columns")
for col in fake.columns:
    print(col, end="\t")

True columns
title	text	date	
Fake columns
title	text	date	

### **1.1** Add new column  <font color = red>[3 marks]</font> <br>

Add new column `news_label` to both the DataFrames and assign labels

In [14]:
# Add a new column 'news_label' to the true news DataFrame and assign the label "1" to indicate that these news are true

true["news_label"] = 1
# Add a new column 'news_label' to the fake news DataFrame and assign the label "0" to indicate that these news are fake
fake["news_label"] = 0

In [15]:
display_info(true)

title  \
0                In Republican state, Obama talks with worried American mom   
1  South Korea calls for 'bone-numbing' sanctions on North for nuclear test   
2     U.S. to expand refugee program for Central Americans fleeing violence   
3    Bid to mine more coal on U.S. federal lands tests Obama's green agenda   
4              For Republican Rubio, a moment of truth in race to lead U.S.   
5                       Cheesy puns abound as White House hosts online chat   
6            New York's Cuomo proposes $145.3 billion budget, caps spending   
7                         Trump looms behind both Obama and Haley speeches    
8         House of Representatives to try again to seek Iran deal clampdown   
9                                      Obama knocks Trump, voices optimism    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       21417 non-null  object        
 1   text        21417 non-null  object        
 2   date        21417 non-null  datetime64[ns]
 3   news_label  21417 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 669.4+ KB


None

### **1.2** Merge DataFrames  <font color = red>[2 marks]</font> <br>

Create a new Dataframe by merging True and Fake DataFrames

In [16]:
# Combine the true and fake news DataFrames into a single DataFrame
df = pd.concat([true, fake], axis=0, ignore_index=True)

In [17]:
# Display the first 5 rows of the combined DataFrame to verify the result
display_info(df)

title  \
0                In Republican state, Obama talks with worried American mom   
1  South Korea calls for 'bone-numbing' sanctions on North for nuclear test   
2     U.S. to expand refugee program for Central Americans fleeing violence   
3    Bid to mine more coal on U.S. federal lands tests Obama's green agenda   
4              For Republican Rubio, a moment of truth in race to lead U.S.   
5                       Cheesy puns abound as White House hosts online chat   
6            New York's Cuomo proposes $145.3 billion budget, caps spending   
7                         Trump looms behind both Obama and Haley speeches    
8         House of Representatives to try again to seek Iran deal clampdown   
9                                      Obama knocks Trump, voices optimism    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44956 entries, 0 to 44955
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       44916 non-null  object
 1   text        44916 non-null  object
 2   date        44896 non-null  object
 3   news_label  44956 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


None

### **1.3** Handle the null values  <font color = red>[2 marks]</font> <br>

Check for null values and handle it by imputation or dropping the null values

In [19]:
# Check Presence of Null Values
np.sum(df.isna().all())

np.int64(0)

In [20]:
# Handle Rows with Null Values
df.dropna(inplace=True)

### **1.4** Merge the relevant columns and drop the rest from the DataFrame  <font color = red>[3 marks]</font> <br>

Combine the relevant columns into a new column `news_text` and then drop irrelevant columns from the DataFrame

In [21]:
# Combine the relevant columns into a new column 'news_text' by joining their values with a space
df_eda = df.copy()
df_eda["news_text"] = df_eda["title"].add(" ").add(df_eda["text"])
# Drop the irrelevant columns from the DataFrame as they are no longer needed
df_eda.drop(["title", "text"], axis=1, inplace=True)
# Display the first 5 rows of the updated DataFrame to check the result
df_eda.head()

date  news_label  \
0  2016-01-13 00:00:00           1   
1  2016-01-13 00:00:00           1   
2  2016-01-13 00:00:00           1   
3  2016-01-13 00:00:00           1   
4  2016-01-13 00:00:00           1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## **2.** Text Preprocessing <font color = red>[15 marks]</font> <br>






On all the news text, you need to:
<ol type=1>
  <li> Make the text lowercase
  <li> Remove text in square brackets
  <li> Remove punctuation
  <li> Remove words containing numbers
</ol>


Once you have done these cleaning operations you need to perform POS tagging and lemmatization on the cleaned news text, and remove all words that are not tagged as NN or NNS.

### **2.1** Text Cleaning  <font color = red>[5 marks]</font> <br>



#### 2.1.0 Create a new DataFrame to store the processed data




--- Applying Updated Cleaning and Lemmatization ---


date  news_label  \
0  2016-01-13 00:00:00           1   
1  2016-01-13 00:00:00           1   
2  2016-01-13 00:00:00           1   
3  2016-01-13 00:00:00           1   
4  2016-01-13 00:00:00           1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

[nltk_data] Downloading package stopwords to /Users/deven/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [27]:
# Apply the lemmatization function to the cleaned text to create 'lemmatized_news_text'

  0%|          | 0/44896 [00:00<?, ?it/s]

date  news_label  \
0  2016-01-13 00:00:00           1   
1  2016-01-13 00:00:00           1   
2  2016-01-13 00:00:00           1   
3  2016-01-13 00:00:00           1   
4  2016-01-13 00:00:00           1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

date  news_label  \
44946  2016-01-20 00:00:00           0   
44947  2016-01-19 00:00:00           0   
44948  2016-01-19 00:00:00           0   
44949  2016-01-19 00:00:00           0   
44950  2016-01-18 00:00:00           0   
44951  2016-01-16 00:00:00           0   
44952  2016-01-16 00:00:00           0   
44953  2016-01-15 00:00:00           0   
44954  2016-01-14 00:00:00           0   
44955  2016-01-12 00:00:00           0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

#### 2.1.1 Write the function to clean the text and remove all the unnecessary elements  <font color = red>[4 marks]</font> <br>



In [60]:
# Write the function here to clean the text and remove all the unnecessary elements

# Convert to lower case

# Remove text in square brackets

# Remove punctuation


# Remove words with numbers
def clean_text(text):
    """
    Cleans the input text by:
    - Lowercasing
    - Removing punctuation (includes square brackets)
    - Removing words containing numbers
    - Removing extra whitespace
    """
    text = str(text).lower()  # Converting to string and lowercasing
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Removing punctuations

    # Removing words that contain any digits
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Removing punctuations

    # Removing words that contain any digits
    text = re.sub(
        r"\b\w*\d\w*\b", "", text
    )  # \b word boundary, \w* zero or more word chars, \d a digit

    text = re.sub(
        r"\s+", " ", text
    ).strip()  # Removing extra spaces and strip leading/trailing
    return text


print("\n--- Applying Updated Cleaning and Lemmatization ---")

# Apply the cleaning function to create 'cleaned_news_text'

#### 2.1.2  Apply the function to clean the news text and store the cleaned text in a new column within the new DataFrame. <font color = red>[1 mark]</font> <br>


In [61]:
# Apply the function to clean the news text and remove all unnecessary elements
# Store it in a separate column in the new DataFrame
processed_df = df_eda.copy()
processed_df["cleaned_news_text"] = processed_df["news_text"].apply(clean_text)

### **2.2** POS Tagging and Lemmatization  <font color = red>[10 marks]</font> <br>



#### 2.2.1 Write the function for POS tagging and lemmatization, filtering stopwords and keeping only NN and NNS tags <font color = red>[8 marks]</font> <br>



In [62]:
# Write the function for POS tagging and lemmatization, filtering stopwords and keeping only NN and NNS tags

import nltk
from nltk.corpus import stopwords
from tqdm.notebook import tqdm

# Download the stopwords corpus
nltk.download("stopwords")

# Load spacy model
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])
stop_words = set(stopwords.words("english"))


def pos_tag_lemmatize(texts):
    """
    Apply POS-based lemmatization to a list/series of texts.
    Only nouns (NN, NNS) are kept, lemmatized and lowercased.
    """
    lemmatized_texts = []

    # Batch processing using spaCy's pipe
    for doc in tqdm(nlp.pipe(texts, batch_size=50, n_process=1), total=len(texts)):
        tokens = []
        for token in doc:
            if (
                (token.tag_ in ["NN", "NNS"])
                and (token.text.lower() not in stop_words)
                and token.is_alpha
            ):
                tokens.append(token.lemma_.lower())
        lemmatized_texts.append(" ".join(tokens))

    return lemmatized_texts

#### 2.2.2  Apply the POS tagging and lemmatization function to cleaned text and store it in a new column within the new DataFrame. <font color = red>[2 mark]</font> <br>

**NOTE: Store the cleaned text and the lemmatized text with POS tags removed in separate columns within the new DataFrame.**

**This will be useful for analysing character length differences between cleaned text and lemmatized text with POS tags removed during EDA.**


In [63]:
# Apply POS tagging and lemmatization function to cleaned text
# Store it in a separate column in the new DataFrame
processed_df["lemmatized_news_text"] = pos_tag_lemmatize(
    processed_df["cleaned_news_text"].tolist()
)


processed_df.head()

### Save the Cleaned data as a csv file (Recommended)

In [34]:
## Recommended to perform the below steps to save time while rerunning the code
processed_df.to_csv("clean_df.csv", index=False)
df_clean = pd.read_csv("clean_df.csv")

In [35]:
# Check the first few rows of the DataFrame
df_clean.head(10)

date  news_label  \
0  2016-01-13 00:00:00           1   
1  2016-01-13 00:00:00           1   
2  2016-01-13 00:00:00           1   
3  2016-01-13 00:00:00           1   
4  2016-01-13 00:00:00           1   
5  2016-01-13 00:00:00           1   
6  2016-01-13 00:00:00           1   
7  2016-01-13 00:00:00           1   
8  2016-01-13 00:00:00           1   
9  2016-01-13 00:00:00           1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [36]:
# Check the dimensions of the DataFrame
df_clean.shape

(44896, 5)

In [45]:
# Check the number of non-null entries and data types of each column
df.isna().sum()

title         0
text          0
date          0
news_label    0
dtype: int64

## **3.** Train Validation Split <font color = red>[5 marks]</font> <br>

In [68]:
# Import Train Test Split and split the DataFrame into 70% train and 30% validation data

## **4.** Exploratory Data Analysis on Training Data  <font color = red>[40 marks]</font> <br>

Perform EDA on cleaned and preprocessed texts to get familiar with the training data by performing the tasks given below:

<ul>
  <li> Visualise the training data according to the character length of cleaned news text and lemmatized news text with POS tags removed
  <li> Using a word cloud, find the top 40 words by frequency in true and fake news separately
  <li> Find the top unigrams, bigrams and trigrams by frequency in true and fake news separately
</ul>





### **4.1** Visualise character lengths of cleaned news text and lemmatized news text with POS tags removed  <font color = red>[10 marks]</font> <br>



##### 4.1.1  Add new columns to calculate the character lengths of the processed data columns  <font color = red>[3 marks]</font> <br>



In [69]:
# Add a new column to calculate the character length of cleaned news text

# Add a new column to calculate the character length of lemmatized news text with POS tags removed

##### 4.1.2  Create Histogram to visualise character lengths  <font color = red>[7 marks]</font> <br>

 Plot both distributions on the same graph for comparison and to observe overlaps and peak differences to understand text preprocessing's impact on text length.

In [70]:
# Create a histogram plot to visualise character lengths

# Add histogram for cleaned news text

# Add histogram for lemmatized news text with POS tags removed

### **4.2** Find and display the top 40 words by frequency among true and fake news in Training data after processing the text  <font color = red>[10 marks]</font> <br>



##### 4.2.1 Find and display the top 40 words by frequency among true news in Training data after processing the text  <font color = red>[5 marks]</font> <br>

In [71]:
## Use a word cloud find the top 40 words by frequency among true news in the training data after processing the text

# Filter news with label 1 (True News) and convert to it string and handle any non-string values

# Generate word cloud for True News

##### 4.2.2 Find and display the top 40 words by frequency among fake news in Training data after processing the text  <font color = red>[5 marks]</font> <br>

In [72]:
## Use a word cloud find the top 40 words by frequency among fake news in the training data after processing the text

# Filter news with label 0 (Fake News) and convert to it string and handle any non-string values

# Generate word cloud for Fake News

### **4.3** Find and display the top unigrams, bigrams and trigrams by frequency in true news and fake news after processing the text  <font color = red>[20 marks]</font> <br>




##### 4.3.1 Write a function to get the specified top n-grams  <font color = red>[4 marks]</font> <br>



In [73]:
# Write a function to get the specified top n-grams

##### 4.3.2 Handle the NaN values  <font color = red>[1 mark]</font> <br>



In [74]:
# Handle NaN values in the text data

### For True News




##### 4.3.3 Display the top 10 unigrams by frequency in true news and plot them as a bar graph  <font color = red>[2.5 marks]</font> <br>

In [75]:
# Print the top 10 unigrams by frequency in true news and plot the same using a bar graph

##### 4.3.4 Display the top 10 bigrams by frequency in true news and plot them as a bar graph  <font color = red>[2.5 marks]</font> <br>



In [76]:
# Print the top 10 bigrams by frequency in true news and plot the same using a bar graph

##### 4.3.5 Display the top 10 trigrams by frequency in true news and plot them as a bar graph  <font color = red>[2.5 marks]</font> <br>



In [77]:
# Print the top 10 trigrams by frequency in true news and plot the same using a bar graph

### For Fake News







##### 4.3.6 Display the top 10 unigrams by frequency in fake news and plot them as a bar graph  <font color = red>[2.5 marks]</font> <br>

In [78]:
# Print the top 10 unigrams by frequency in fake news and plot the same using a bar graph

##### 4.3.7 Display the top 10 bigrams by frequency in fake news and plot them as a bar graph  <font color = red>[2.5 marks]</font> <br>



In [79]:
# Print the top 10 bigrams by frequency in fake news and plot the same using a bar graph

##### 4.3.8 Display the top 10 trigrams by frequency in fake news and plot them as a bar graph  <font color = red>[2.5 marks]</font> <br>



In [80]:
# Print the top 10 trigrams by frequency in fake news and plot the same using a bar graph

## **5.** Exploratory Data Analysis on Validation Data [Optional]

Perform EDA on validation data to differentiate EDA on training data with EDA on validation data and the tasks are given below:

<ul>
  <li> Visualise the data according to the character length of cleaned news text and lemmatized text with POS tags removed
  <li> Using a word cloud find the top 40 words by frequency in true and fake news separately
  <li> Find the top unigrams, bigrams and trigrams by frequency in true and fake news separately
</ul>





### **5.1** Visualise character lengths of cleaned news text and lemmatized news text with POS tags removed

##### 5.1.1  Add new columns to calculate the character lengths of the processed data columns

In [81]:
# Add a new column to calculate the character length of cleaned news text

# Add a new column to calculate the character length of lemmatized news text with POS tags removed

##### 5.1.2  Create Histogram to visualise character lengths

Plot both distributions on the same graph for comparison and to observe overlaps and peak differences to understand text preprocessing's impact on text length.

In [82]:
# Create a histogram plot to visualise character lengths

# Add histogram for cleaned news text

# Add histogram for lemmatized news text with POS tags removed

### **5.2** Find and display the top 40 words by frequency among true and fake news after processing the text

##### 5.2.1  Find and display the top 40 words by frequency among true news in validation data after processing the text

In [83]:
## Use a word cloud find the top 40 words by frequency among true news after processing the text

# Generate word cloud for True News

##### 5.2.2  Find and display the top 40 words by frequency among fake news in validation data after processing the text

In [84]:
## Use a word cloud find the top 40 words by frequency among fake news after processing the text

# Generate word cloud for Fake News

### **5.3** Find and display the top unigrams, bigrams and trigrams by frequency in true news and fake news after processing the text  





##### 5.3.1 Write a function to get the specified top n-grams

In [85]:
## Write a function to get the specified top n-grams

##### 5.3.2 Handle the NaN values

In [86]:
## First handle NaN values in the text data

### For True News



##### 5.3.3 Display the top 10 unigrams by frequency in true news and plot them as a bar graph

In [87]:
## Print the top 10 unigrams by frequency in true news and plot the same using a bar graph

##### 5.3.4 Display the top 10 bigrams by frequency in true news and plot them as a bar graph

In [88]:
## Print the top 10 bigrams by frequency in true news and plot the same using a bar graph

##### 5.3.5 Display the top 10 trigrams by frequency in true news and plot them as a bar graph

In [89]:
## Print the top 10 trigrams by frequency in true news and plot the same using a bar graph

### For Fake News

##### 5.3.6 Display the top 10 unigrams by frequency in fake news and plot them as a bar graph

In [90]:
## Print the top 10 unigrams by frequency in fake news and plot the same using a bar graph

##### 5.3.7 Display the top 10 bigrams by frequency in fake news and plot them as a bar graph

In [91]:
## Print the top 10 bigrams by frequency in fake news and plot the same using a bar graph

##### 5.3.8 Display the top 10 trigrams by frequency in fake news and plot them as a bar graph

In [92]:
## Print the top 10 trigrams by frequency in fake news and plot the same using a bar graph

## **6.** Feature Extraction  <font color = red>[10 marks]</font> <br>

For any ML model to perform classification on textual data, you need to convert it to a vector form. In this assignment, you will use the Word2Vec Vectorizer to create vectors from textual data. Word2Vec model captures the semantic relationship between words.


### **6.1** Initialise Word2Vec model  <font color = red>[2 marks]</font>

In [93]:
## Write your code here to initialise the Word2Vec model by downloading "word2vec-google-news-300"

### **6.2** Extract vectors for cleaned news data   <font color = red>[8 marks]</font>

In [94]:
## Write your code here to extract the vectors from the Word2Vec model for both training and validation data


## Extract the target variable for the training data and validation data

## **7.** Model Training and Evaluation <font color = red>[45 marks]</font>

You will use a set of supervised models to classify the news into true or fake.

### **7.0** Import models and evaluation metrics

In [95]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

### **7.1** Build Logistic Regression Model  <font color = red>[15 marks]</font>

##### 7.1.1 Create and train logistic regression model on training data  <font color = red>[10 marks]</font>

In [96]:
## Initialise Logistic Regression model

## Train Logistic Regression model on training data

## Predict on validation data

##### 7.1.2 Calculate and print accuracy, precision, recall and f1-score on validation data <font color = red>[5 marks]</font>

In [97]:
## Calculate and print accuracy, precision, recall, f1-score on predicted labels

In [98]:
# Classification Report

### **7.2** Build Decision Tree Model <font color = red>[15 marks]</font>

##### 7.2.1 Create and train a decision tree model on training data <font color = red>[10 marks]</font>

In [99]:
## Initialise Decision Tree model

## Train Decision Tree model on training data

## Predict on validation data

##### 7.2.2 Calculate and print accuracy, precision, recall and f1-score on validation data <font color = red>[5 marks]</font>

In [100]:
## Calculate and print accuracy, precision, recall, f1-score on predicted labels

In [101]:
# Classification Report

### **7.3** Build Random Forest Model <font color = red>[15 marks]</font>


##### 7.3.1 Create and train a random forest model on training data <font color = red>[10 marks]</font>

In [102]:
## Initialise Random Forest model

## Train Random Forest model on training data

## Predict on validation data

 ##### 7.3.2 Calculate and print accuracy, precision, recall and f1-score on validation data <font color = red>[5 marks]</font>

In [103]:
## Calculate and print accuracy, precision, recall, f1-score on predicted labels

In [104]:
# Classification Report

## **8.** Conclusion <font color = red>[5 marks]</font>

Summarise your findings by discussing patterns observed in true and fake news and how semantic classification addressed the problem. Highlight the best model chosen, the evaluation metric prioritised for the decision, and assess the approach and its impact.